In [1]:
%%writefile data_preproc.py
from tqdm.notebook import tqdm
from itertools import chain
import json
import re
from app_utils import to_start, rid_blank, preproc_num, to_normal
from trns.preproc_En import pre_en, preproc_en
from trns.preproc_kor import preproc_ko2en


def preProc(X, lang, to_start, pre_ko, preproc_en, en_vocs):
    
    #X = to_start(X)
    
    if lang == 'en':
        #X = pre_en(X)
        X = preproc_en(X,en_vocs)
        X = preproc_num(X)
       
    else:
        X = pre_ko.forward(X)
        X = preproc_num(X) 

    return X 


def preProc_save(X, step_num, lang,to_start, pre_ko, preproc_en, en_vocs,f_toSave):
    
    #n_iter = len(X) // step_num +1
    n_iter = len(X) // step_num + min((len(X) % step_num), 1) * 1

    for i in tqdm(range(n_iter)):
        Xsub = X[i*step_num:(i+1)*step_num]
        #Y = list(chain(*[preProc(s, 'en', to_start, pre_ko, preproc_en, en_vocs) for s in Xsub]))
        Y = preProc(Xsub, lang, to_start, pre_ko, preproc_en, en_vocs)
        with open(f_toSave, 'w' if i==0 else 'a') as f:
            f.write('\n'.join(Y)+'\n')
    return Y

#with open('to_preproc/pre_processed_short05.en', 'r') as f:
#    X = f.read().split('\n')[:100]


def sanitize_input(in_file):
    
    p = re.compile('[\`\^]')
    p1 = re.compile('\`')
    p2 = re.compile('\^')
    
    with open(in_file, 'r') as f:
        X = f.read().split('\n')
        X = [s+'.' for s in X]

    if len([s for s in X if p.search(s) is not None])>0:
        print("`^ id detected !!!!")
        print([(i,s) for i,s in enumerate(X) if p.search(s) is not None])

    X = [p2.sub('ˆ',p1.sub("'",s)) for s in X] #if p.search(s) is None]
    
    return X

pre_ko = preproc_ko2en()
en_vocs = pre_en()

path0 = 'innovators/'
f_toSave = path0+'new_output/dev8.'
step_num = 10000

X = sanitize_input(path0+'Xen')
Y = preProc_save(X, step_num, 'en', to_start, pre_ko, preproc_en, en_vocs,f_toSave+'en')

X = sanitize_input(path0+'Xko')
Y = preProc_save(X, step_num, 'ko', to_start, pre_ko, preproc_en, en_vocs,f_toSave+'ko')

"""
with open('innovators/Xen', 'r') as f:
    X = f.read().split('\n')
    X = [s+'.' for s in X]
    
if len([s for s in X if p.search(s) is not None])>0:
    print("`^ id detected !!!!")
    print([(i,s) for i,s in enumerate(X) if p.search(s) is not None])
    
X = [p2.sub('ˆ',p1.sub("'",s)) for s in X] #if p.search(s) is None]
"""  

"""
step_num = 10000
n_iter = len(X) // step_num + min((len(X) % step_num), 1) * 1
for i in tqdm(range(n_iter)):
    Xsub = X[i*step_num:(i+1)*step_num]
    #Y = list(chain(*[preProc(s, 'en', to_start, pre_ko, preproc_en, en_vocs) for s in Xsub]))
    Y = preProc(Xsub, 'en', to_start, pre_ko, preproc_en, en_vocs)
    with open('innovators/dev8.en', 'w' if i==0 else 'a') as f:
        f.write('\n'.join(Y)+'\n')


with open('innovators/Xko', 'r') as f:
    X = f.read().split('\n')
    
if len([s for s in X if p.search(s) is not None])>0:
    print("`^ id detected !!!!")
    print([(i,s) for i,s in enumerate(X) if p.search(s) is not None])
    
X = [p2.sub('ˆ',p1.sub("'",s)) for s in X] #if p.search(s) is None]
"""


"""
step_num = 10000
n_iter = len(X) // step_num + min((len(X) % step_num), 1) * 1
for i in tqdm(range(n_iter)):
    Xsub = X[i*step_num:(i+1)*step_num]
    #Y = list(chain(*[preProc(s, 'en', to_start, pre_ko, preproc_en, en_vocs) for s in Xsub]))
    Y = preProc(Xsub, 'ko', to_start, pre_ko, preproc_en, en_vocs)
    with open('innovators/dev8.ko', 'w' if i==0 else 'a') as f:
        f.write('\n'.join(Y)+'\n')
"""

len(Y)


Writing data_preproc.py


In [ ]:
with open('ted_data/to_train/ted_train.en', 'r') as f:
    X = f.read().split('\n')

with open('ted_data/to_train/ted_train.ko', 'r') as f:
    Y = f.read().split('\n')

In [57]:
len(X),len(Y)

(166213, 166213)

In [59]:
X[-3:], Y[-3:]

(["^ and _ the _ pin head ^ i've _ made _ green _ around _ there _ by _ scrap ing _ the _ particle s _ off _ a _ green _ shirt _ and _ then _ pressed _ onto _ the _ needle _ .",
  "^ it's _ very _ painstaking _ work _ , _ but _ the _ best _ thing s _ come _ in _ small _ package s _ .",
  '_ ( ^ laughter _ ) ^ bruno ^ gi us sani _ : ^ willard ^ wigan _ !'],
 ['_ 그리 고 _ 이 _ 못 _ 머리 의 _ 초록색 은 _ 초록색 _ 셔츠 의 _ 입자 를 _ 긁 어내 어 _ 못 _ 머리 에 _ 눌러 주 었습니다 _ .',
  '_ 제 _ 작품 _ 활동 은 _ 꽤 나 _ 공 이 _ 들어가 는 _ 일 이 ㅂ니다 . _ 하 지만 _ 최고 의 _ 것 들 은 _ 항 상 _ 작 은 _ 것 에서 부터 _ 비롯 되는 _ 거 니까 요 _ .',
  '_ ( _ 웃음 _ ) _ 부르 노 _ 지우 사 니 _ : _ 윌 라 드 _ 위 간 _ 씨 이 ㅓㅆ 습니다 _ ! _'])

In [34]:
ord('ˆ')

710

In [36]:
"""
import re
p = re.compile("\([^\)]*[^0-9\~\-\–\s\.\,\;\:\%가-힣a-zA-Z]+[^\(]*\)")
q = re.compile('^\(')
z = re.compile('\s+')
z1 = re.compile(r'\\')
"""

In [56]:
with open('ted_data/to_train/ted_train.en', 'r') as f:
    X = f.read().split('\n')

with open('ted_data/to_train/ted_train.ko', 'r') as f:
    Y = f.read().split('\n')

In [42]:
Y[:100]

['_ _ 이쪽 은 ^ bill ^ lange _ 이 고 _ , _ 저 는 ^ david ^ gallo 이 ㅂ니다',
 '_ 우리 는 _ 여러분 에게 _ 바닷속 _ 이야기 를 _ 영상 과 _ 함께 _ 들리 ㅓ 주 고자 _ 하 ㅂ니다 _ .',
 '_ 저희 는 _ 끝내 주는 _ 타이 타 닉 _ 비디오 도 _ 있 기 ㄴ _ 하 ㅂ니다만 _ 뭐 .. 여기 서는 _ 눈 꼽 만큼 도 _ 보여주 ㄹ _ 생각 이 없 습니다 _ .',
 '_ _ 비 록 _ 타이 타 닉 이 _ 박스오피스 에서 _ 굉장 한 _ 실적 을 _ 거두 기 ㄴ _ 했 지만 _ 바다 가 _ 들리 ㅓ 주는 _ 이야기 _ 중 _ 가장 _ 재 밌 는 _ 것 은 _ 아니 ㅂ니다 _ .',
 '_ 문제 라면 _ 우리 는 _ 우리 가 _ 바다 를 _ 이미 _ 알고있 다고 _ 믿 는 거 죠 _ .',
 '_ 보통 _ 바다 가 _ 지구 의 _ 75 _ % _ 를 _ 감싸 고 _ 있 다는 _ 것 _ , _',
 '_ 지구상 _ 바다 의 _ 대부분 은',
 '_ 평균 _ 깊이 가 _ 2 마일 _ _ 정도 _ 된다 는 걸 _ 알고있 죠',
 '_ 그리 고 는 _ 해변 가 에 _ 가 거나 _ 이러 ㄴ _ 바닷가 의 _ 이미지 를 _ 보고 는 _ 바다 란 _ 그냥 _ 거대 한 _ 파란색 의 _ 아르 ㄴ 하게 _ 넘 실 거리 는 _ 파도 와 _ 조수 간 만 의 _ 차 가 있 다 _ 정도 만 _ 생각 하 죠 _ 사실 _ 그 _ 속 에 _ 무엇 이 _ 있 는 지는 _ 상상 도 _ 못하 는 거 죠 .. _ .',
 '_ 바닷속 에는 _ 지구상 _ 가장 _ 기 ㄴ _ 산맥 과',
 '_ 대부분 의 _ 생명체 들이 _ 있 고',
 '_ 대부분 의 _ 지진 과 _ 화산 활동 이 _ 바닷속 에는 _ 일어나 ㅂ니다 . _ 저 _ 바다 _ 밑 _ 바닥 에는',
 '_ 밀림 보다 도 _ 더 _ 많 은 _ 종류 의 _ 생명체 들이 _ 훨 씬 _ 높 은 _ 밀집 도 를 _ 형성 하 며 _ 살 고 _ 있 습니다 _ .',
 '_ 이러 ㄴ _ 친밀감 을

In [39]:
mX = [z.sub(' ',p.sub('',q.sub('',s))).strip() for s in X][:-1]
mY = [z.sub(' ',p.sub('',q.sub('',s))).strip() for s in Y][:-1]

In [55]:
with open('ted_data/to_train/ted_train.en', 'w') as f:
    f.write('\n'.join(X[:166213]))

with open('ted_data/to_train/ted_train.ko', 'w') as f:
    f.write('\n'.join(Y[:166213]))

In [140]:
nX = [s for s in mY if z1.search(s) is not None]
len(nX), nX

(0, [])

In [136]:
aa = "일본 , \ 인도 , 미국 , 프랑스 ,   \'s 200b \u200b 오스트레일리아 "
z1.search(aa)

<re.Match object; span=(5, 6), match='\\'>

In [146]:
len(mX)

669

In [32]:
len(X), len(Y)

(671, 671)

In [65]:
kkk = 444   #512  #233('\'s')   #247    #345
X[kkk],Y[kkk]

('^ mechanical _ ventilation _ , _ or _ assist ed _ ventilation _ , _ is _ the _ medical _ term _ for _ artificial _ ventilation _ where _ mechanical _ mean s _ are _ used _ to _ assist _ or _ replace _ spontaneous _ breath ing _ .',
 '_ 기계 환기 _ ( _ 機 _ 械 _ 換 _ 氣 _ , _ mechanical _ ventilation _ ) _ 은 _ 자발적 이 ㄴ _ 호흡 을 _ 보조 하 거나 _ 대체 하기 _ 위해 _ 기계적 _ 수단 이 _ 사용 되는 _ 인공 환기 이 다 _ .')

In [53]:
import re
p = re.compile('[\`\^]')
Xsbol = [s for s in X if p.search(s) is None]

In [54]:
len(Xsbol)

1295819

In [ ]:
X[-10:]

In [ ]:
Y[-10:]

In [16]:
with open('to_preproc/totrain_en_EL', 'r') as f:
    X = f.read().split('\n')[:-2]

with open('to_preproc/totrain_ko_EL', 'r') as f:
    Y = f.read().split('\n')[:-2]
    
mX = [s for i,s in enumerate(X) if i%31 not in [7,17]]
dX1 = [s for i,s in enumerate(X) if i%31 == 7]
tX1 = [s for i,s in enumerate(X) if i%31 == 17]
mY = [s for i,s in enumerate(Y) if i%31 not in [7,17]]
dY1 = [s for i,s in enumerate(Y) if i%31 == 7]
tY1 = [s for i,s in enumerate(Y) if i%31 == 17]

with open('to_preproc/train4.en', 'w') as f:
    f.write('\n'.join(mX))

with open('to_preproc/train4.ko', 'w') as f:
    f.write('\n'.join(mY))

In [17]:
with open('to_preproc/totrain_en_talk', 'r') as f:
    X = f.read().split('\n')[:-2]

with open('to_preproc/totrain_ko_talk', 'r') as f:
    Y = f.read().split('\n')[:-2]
    
mX = [s for i,s in enumerate(X) if i%5000 not in [7,17]]
dX2 = [s for i,s in enumerate(X) if i%5000 == 7]
tX2 = [s for i,s in enumerate(X) if i%5000 == 17]
mY = [s for i,s in enumerate(Y) if i%5000 not in [7,17]]
dY2 = [s for i,s in enumerate(Y) if i%5000 == 7]
tY2 = [s for i,s in enumerate(Y) if i%5000 == 17]

with open('to_preproc/train5.en', 'w') as f:
    f.write('\n'.join(mX))

with open('to_preproc/train5.ko', 'w') as f:
    f.write('\n'.join(mY))

In [19]:
dX = dX1 + dX2
dY = dY1 + dY2
tX = tX1 + tX2
tY = tY1 + tY2
len(dX), len(dY), dX[:2], dY[:2]

(42,
 42,
 ['^ the ^ ocean ^ viking _ , _ a _ rescue _ ship _ operat ed _ by ` msf _ and ` sos ^ medi tera nee _ that _ was _ carrying _ almost _ 3 00 _ migrant s _ , _ was _ quarantin ed _ for _ 14 _ days _ in ^ po zz allo _ , ^ sicily _ .',
  '^ corticosteroid s _ by _ mouth _ improve _ the _ chance _ of _ recovery _ and _ decreas e _ the _ overall _ duration _ of _ symptoms _ .'],
 ['` msf 와 ` sos _ 메 디 테 라 네 가 _ 운영 하는 _ 구조 선인 _ 오 션 _ 바이킹 은 _ 시칠리아 의 _ 포 잘 로 에서 _ 14 일 _ 간 _ 격리 되었 다 _ .',
  '_ 구강 _ 코르티코스테로이드 는 _ 회복 의 _ 가능성 을 _ 높이 고 _ 증상 의 _ 지속 기간 을 _ 줄이 ㄴ 다 _ .'])

In [20]:
with open('to_preproc/dev_toAdd.en', 'w') as f:
    f.write('\n'.join(dX))

with open('to_preproc/dev_toAdd.ko', 'w') as f:
    f.write('\n'.join(dY))

with open('to_preproc/test_toAdd.en', 'w') as f:
    f.write('\n'.join(tX))

with open('to_preproc/test_toAdd.ko', 'w') as f:
    f.write('\n'.join(tY))